# Migrating to 0.2.0

The 0.2.0 release contains a handful of breaking changes compared to the previous 0.1.x releases.  This guide will list out these changes as well as how to migrate to the new features that encompass them in 0.2.x.

## Pydantic Support

### Using Pydantic Models With Guardrails

#### Applying Guardrails Validations Fields In A Pydantic Model

## Polymorphism With Discriminators In Output Schemas

## Changes To Validators (Note for Raf: This section title should be more specific)

## Removal Of Script Support

## String Formatting In Prompts
Previously Guardrails used `f-string`'s and `str.format` to include prompt variables, prompt primitives, and other partials in the prompt sent to the LLM.  This necessitated any braces, aka curly brackets, to be escaped with additional braces to prevent its contents from being considered as variable that should be substituted during string formatting.

We now use `Template.safe_substitute` to avoid this issue completely.  This does, however, require a change to how variables are expressed in string arguments such as prompts and instructions.  Now instead of expressing a variable for the prompt as `{{my_var}}`, it should be expressed as `${my_var}`.

In addition to this change in variable syntax, we have also started namespacing our prompt primitives.  Whereas before you would use one of our built in primitives like so: `@complete_json_suffix_v2`, they should now be specified as `${gr.complete_json_suffix_v2}`.

To highlight these changes in practice, below is the prompt section of a RAIL spec where each tab shows the respective before and after format.

=== "0.1.x"

```xml
<prompt>
Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 'None'.

{document}

@xml_prefix_prompt

{{output_schema}}


@json_suffix_prompt
</prompt>
```

=== "0.2.x
```xml
<prompt>
Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 'None'.

${document}

${gr.xml_prefix_prompt}

${output_schema}

${gr.json_suffix_prompt}
</prompt>
```